# The Solow Model with Alternative Specifications for Human Capital

This project aims to solve the Solow Model with Alternative Specifications for Human Capital. 
To accomplish this, I present the Solow Model with Technology and extend the model to account for human capital.
With the extended model, I show both analytical and numerical solutions. I also visualize the results and analyze their dependence on the model's parameters.
Having examined the model, I explore an alternative specification for human capital, considering a model with distinct groups of workers, one highly educated and the other unskilled.


Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm
import random

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

## The Solow Model

Introducing the Solow Model...

## The Solow Model with Human Capital

I try to solve for the steady state of capital and human capital using the Solow-equations and `SymPy`. <br>
To solve for the steady state, I define the parameters as symbols, formulate steady-state Solow equations, and use `SymPy.solve`.

In [2]:
# Creates a SymPy symbol for each variable and parameter
k, h, s_K, s_H, n, g, delta, alpha, varphi = sm.symbols('k, h, s_K, s_H, n, g, delta, alpha, varphi')

# Sets the Solow equations in steady state as SumPy equations
ss_solow_k = sm.Eq(0, (1 / ((1 + n) * (1 + g))) * (s_K * k**alpha * h**varphi - (n + g + delta + n * g) * k))
ss_solow_h = sm.Eq(0, (1 / ((1 + n) * (1 + g))) * (s_H * k**alpha * h**varphi - (n + g + delta + n * g) * h))

# Try to solve the systems of equations algebraically for k and h using SumPy
try:
    ss_k_h = sm.solve([ss_solow_k, ss_solow_h], k, h)
except (NotImplementedError):
    print('Unable to solve the system of equations algebraically.')

Unable to solve the system of equations algebraically.


Due to the limitations of `SymPy` in solving the given system of equations algebraically, I use an alternative approach. I can find the solutions by utilizing the steady-state equations for capital and human capital as defined in "Introducing Advanced Macroeconomics" by Sørensen and Whitta-Jacobsen. This involves creating lambdified expressions for the equations and using them to compute the steady-state values of capital and human capital.

In [3]:
# Sets the steady state equations
ss_eq_k = ((s_K**(1-varphi) * s_H**varphi) / (n + g + delta + n * g))**(1 / (1 - alpha - varphi))
ss_eq_h = ((s_K**(alpha) * s_H**(1-alpha)) / (n + g + delta + n * g))**(1 / (1 - alpha - varphi))

# Creates a function for the steady state of k and h using SumPy lambdify
ss_func = sm.lambdify(args = (s_K, s_H, n, g, delta, alpha, varphi), expr = (ss_eq_k, ss_eq_h))

I set the name of the parameter values to the name of each parameter plus `_val` to not overwrite the original values.

In [4]:
# Sets parameter values
s_K_val = 0.3
s_H_val = 0.2
n_val = 0.02
g_val = 0.02
delta_val = 0.1
alpha_val = 1/3
varphi_val = 1/3

Given the parameter values, I find the steady state values of capital and human capital.

In [5]:
# Calls each steady state function for the parameter values
ss_k_func, ss_h_func = ss_func(s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val)

# Prints the steady state value for capital and human capital
print(f'Analytical solution:\n\
There are {ss_k_func:.3f} units of capital, and {ss_h_func:.3f} units of human capital in steady state.')

Analytical solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state.


Now, that I have an analytical solution independent of any algorithm's initial values or tolerance, I can employ an optimizer to determine the steady state values for capital and human capital and compare the results to the analytical solution. 

In [6]:
# Call function to find the steady state of capital and human capital
from modelproject import n_ss

# To see the function delete # below
#help(n_ss)

# Use SciPy optimize to find roots
# Guess on values
initial_guess = [2, 2]

# Solve the function for steady state
sol = optimize.root(fun = n_ss, x0 = initial_guess, args = (s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val), method = 'hybr', tol = 0.0001)

# Save the values
num_ss_k, num_ss_h = sol.x

# Print the steady-state values for capital and human capital
print(f'Numerical Solution:\n\
There are {num_ss_k:.3f} units of capital, and {num_ss_h:.3f} units of human capital in steady state')

Numerical Solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state


The solutions to the optimization problem are highly sensitive to the initial guess. To address this issue, I employ a multi-start optimization technique.
The multi-start optimization takes a specified number of random initial guesses within a bounded range for the variables. For each guess, it solves for the steady-state variables, calculates the residual of the function, and stores both the solution and the residual. Finally, it returns the solution with the smallest residual, corresponding to the best convergence or minimum error among all the initial guesses.
This approach improves the reliability of the optimization results by exploring multiple starting points and selecting the solution that provides the best convergence.


In [7]:
# Call the Multi-Start function to find the steady state of capital and human capital
from modelproject import multi_start

multi_start(args = (s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val))


Muti-Start Solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state
The functions residual is 1.0671117114813116e-16


Given an initial guess with values which are close to the analytical solution the optimizer is very precise, but if the initial guess are too far from the analytical solution the optimizer does not converge to ammend this issue I e

I need to make a callback in optimize root where I check for which values the numerical solution converges. 

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

In [8]:
import numpy as np
from scipy import optimize

# Number of random initial guesses
num_guesses = 200

# Define bounds for capital and human capital variables
bounds = [(0.1, 20), (0.1, 20)]

# Create a list to store solutions and their residuals
solutions = []

# Loop through each random initial guess
for _ in range(num_guesses):
    # Generate a random initial guess within bounds
    initial_guess = [np.random.uniform(low=bound[0], high=bound[1]) for bound in bounds]

    # Solve the function for steady state
    sol = optimize.root(fun=n_ss, x0=initial_guess, args=(s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val), method='hybr')

    # Store the solution and its residual in the solutions list
    solutions.append((sol.x, sol.fun))

# Find the solution with the minimum residual
best_solution = min(solutions, key=lambda x: np.linalg.norm(x[1]))

# Save the values
num_ss_k, num_ss_h = best_solution[0]

# Print the steady-state values for capital and human capital
print(f'Numerical Solution:\n\
There are {num_ss_k:.3f} units of capital, and {num_ss_h:.3f} units of human capital in steady state')

Numerical Solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state


## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 